In [58]:
from pathlib import Path

import json
import pandas as pd

import preprocessing
from feature_engineering import FeatureCreator,PredictionFeatureCreator
# from feature_engineering import PredictionFeatureCreator
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload

UsageError: Line magic function `%autoreload` not found.


In [20]:
#レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()

In [21]:
results_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88363 entries, 0 to 88362
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   race_id      88363 non-null  int64  
 1   horse_id     88363 non-null  int64  
 2   jockey_id    88363 non-null  int64  
 3   trainer_id   88363 non-null  int64  
 4   rank         88363 non-null  int64  
 5   sex          88363 non-null  int64  
 6   age          88363 non-null  int64  
 7   weight       88363 non-null  int64  
 8   weight_diff  88363 non-null  int64  
 9   tansho_odds  88363 non-null  float64
 10  popularity   88363 non-null  int64  
 11  impost       88363 non-null  float64
 12  wakuban      88363 non-null  int64  
 13  umaban       88363 non-null  int64  
dtypes: float64(2), int64(12)
memory usage: 9.4 MB


In [22]:
#重複チェック
results_preprocessed.duplicated(subset=["race_id","horse_id"]).sum()

np.int64(0)

In [23]:
#欠損チェック
results_preprocessed.isnull().sum()

race_id        0
horse_id       0
jockey_id      0
trainer_id     0
rank           0
sex            0
age            0
weight         0
weight_diff    0
tansho_odds    0
popularity     0
impost         0
wakuban        0
umaban         0
dtype: int64

In [24]:
#馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
horse_results_preprocessed

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses
0,2016100158,2024-07-27,12,0.0,3.6,1.0,3,3250,0.0,NaN,13.0
1,2016100158,2024-06-22,10,0.0,4.1,0.0,3,3110,1.0,NaN,14.0
2,2016100158,2024-06-01,9,0.0,1.6,0.0,0,1800,1.0,5.0,10.0
3,2016100158,2024-04-13,7,0.0,4.3,0.0,3,4250,0.0,NaN,12.0
4,2016100158,2023-12-23,6,0.0,4.5,0.0,3,4100,0.0,NaN,12.0
...,...,...,...,...,...,...,...,...,...,...,...
221487,2021106341,2024-05-31,5,5.4,2.5,1.0,0,1500,3.0,NaN,12.0
221488,2021106341,2024-05-06,10,0.0,3.0,1.0,0,1500,0.0,NaN,12.0
221489,2021106341,2024-04-25,7,0.0,1.3,0.0,0,1500,2.0,NaN,12.0
221490,2021106341,2024-04-12,12,0.0,4.0,0.0,0,1500,0.0,NaN,12.0


In [33]:
horse_results_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221492 entries, 0 to 221491
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   horse_id      221492 non-null  int64         
 1   date          221492 non-null  datetime64[ns]
 2   rank          221492 non-null  int64         
 3   prize         221492 non-null  float64       
 4   rank_diff     221185 non-null  float64       
 5   weather       221010 non-null  float64       
 6   race_type     221492 non-null  int64         
 7   course_len    221492 non-null  int64         
 8   ground_state  221481 non-null  float64       
 9   race_class    152059 non-null  float64       
 10  n_horses      221485 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(4)
memory usage: 18.6 MB


In [12]:
horse_id_list = [
    2020102562,
    2020104817
]
horse_results_preprocessed.query("horse_id in @horse_id_list")

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses
52500,2020102562,2024-10-19,11,0.0,1.6,0.0,0,1400,0.0,3.0,16.0
52501,2020102562,2024-08-18,15,0.0,4.2,0.0,0,1800,0.0,5.0,16.0
52502,2020102562,2024-05-19,13,0.0,1.8,1.0,0,1600,0.0,5.0,15.0
52503,2020102562,2024-05-04,12,0.0,1.6,0.0,0,1600,1.0,3.0,13.0
52504,2020102562,2024-01-28,1,800.0,0.0,1.0,0,1600,0.0,2.0,14.0
52505,2020102562,2023-12-23,9,0.0,0.9,0.0,0,1200,0.0,2.0,16.0
52506,2020102562,2023-10-08,6,0.0,0.5,1.0,0,1400,0.0,2.0,15.0
52507,2020102562,2023-09-18,3,200.0,0.4,0.0,0,1200,0.0,2.0,16.0
52508,2020102562,2023-08-27,4,120.0,0.3,0.0,0,1200,0.0,2.0,15.0
52509,2020102562,2023-05-06,1,550.0,0.0,0.0,0,1400,0.0,1.0,16.0


In [30]:
race_info_preprocessed = preprocessing.process_race_info()

In [31]:
race_info_preprocessed

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place
0,202301010101,2023-07-22,1,0.0,1200,0.0,0,1,1
1,202301010102,2023-07-22,0,0.0,1000,0.0,0,1,1
2,202301010103,2023-07-22,0,0.0,1700,0.0,0,1,1
3,202301010104,2023-07-22,1,0.0,1500,0.0,0,1,1
4,202301010105,2023-07-22,0,0.0,1700,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...
6569,202410030808,2024-07-21,0,0.0,1000,0.0,1,2,10
6570,202410030809,2024-07-21,1,0.0,2000,1.0,0,5,10
6571,202410030810,2024-07-21,1,0.0,1200,1.0,0,3,10
6572,202410030811,2024-07-21,1,0.0,1800,0.0,0,5,10


In [27]:
race_info_preprocessed.isnull().sum()

race_id           0
date              0
race_type         0
around          290
course_len       57
weather           5
ground_state      0
race_class        0
place             0
dtype: int64

In [28]:
fc = FeatureCreator()
features = fc.create_features()

In [ ]:
# features.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [ ]:
予測当日の処理

In [34]:
import preprocessing

horse_results_preprocessed = preprocessing.process_horse_results(
    save_filename="horse_results_prediction.csv"
)

In [2]:
%autoreload

UsageError: Line magic function `%autoreload` not found.


In [2]:
# 特徴量作成
pfc = PredictionFeatureCreator()
#馬の過去成績集計は事前に行っておいても良い
pfc.agg_horse_n_races()

NameError: name 'PredictionFeatureCreator' is not defined